# Final Project Testing Q+A and Summarization with LLMs

**Testing Out Q+A and Summarization**
* Start by processing and loading documents into a vector db.
* Try retreving information.



## Setup

In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U flash_attn
!pip install pymupdf4llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#help track which versions of libraries we're using
# !pip list | grep transformers
# !pip list | grep accelerate
# !pip list | grep bitsandbytes
# !pip list | grep datasets

sentence-transformers              3.4.1
transformers                       4.51.1
accelerate                         1.6.0
bitsandbytes                       0.45.5
tensorflow-datasets                4.9.8
vega-datasets                      0.9.0


In [2]:

from transformers import pipeline, BitsAndBytesConfig
import bitsandbytes as bnb
import torch
import random
import pandas as pd
from tqdm import tqdm


In [ ]:
#let's make longer output readable without horizontal scrolling
from pprint import pprint

In [3]:
!pip install -q -U langchain-nomic langchain_community unstructured tiktoken langchainhub chromadb langchain langgraph gpt4all

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.

Now let's get the data we're going to use.

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain.docstore.document import Document
from langchain_community.document_loaders import UnstructuredMarkdownLoader



In [7]:
# small documents path: /mydrive/datasci266/finalProject/references/
from google.colab import drive
drive.mount('/content/drive')
file_path = "drive/MyDrive/datasci266/finalProject/references/NIST.AI.100-1.md"
#user_policy = "AGORA codebook.md"
# user_policy = "drive/MyDrive/datasci266/finalProject/references/CA_AI_Policy_deepseek.md"

#deepseek
file_path1 = "drive/MyDrive/datasci266/AI_Policy_deepseek/"
myStates1 = ['NY', 'CA', 'SD', 'AZ', 'WA', 'FL', 'TX']
fileName1 = "_AI_Policy_deepseek.md"

#claude
file_path2 = "drive/MyDrive/datasci266/dataGov_claude/"
myStates2 = ['WY', 'UT', 'CO', 'NJ', 'ME', 'AR', 'MT']
fileName2 = "_dataGov_Policy_claude.md"

#perplexity
file_path3 = "drive/MyDrive/datasci266/AI_Policy_Perplexity/"
myStates3 = ['HA', 'KS', 'MA', 'NM', 'VA', 'VT']
fileName3 = "_AI_Policy_perplexity.md"

#chatgpt
file_path4 = "drive/MyDrive/datasci266/AI_Policy_ChatGPT/"
myStates4 = ['AK', 'AL', 'CT', 'MN', 'MS', 'ND', 'OK']
fileName4 = "_AI_Policy_chatgpt.md"

# myNames = ['deepseek', 'claude', 'perplexity', 'chatgpt']
# for i in myNames:
#   !touch {i}.txt


def connect(fpath, states, fname, output_file):
  with open(output_file, 'w') as outfile:
    for i in range(len(states)):
      try:
        with open(f'{fpath}{states[i]}{fname}') as infile:
          cont = infile.read()
          outfile.write(cont + '\n\n')
      except:
        print(f"error with {fpath}{states[i]}{fname}")
  return output_file

deepseek = connect(file_path1, myStates1, fileName1, "deepseek.txt")
claude = connect(file_path2, myStates2, fileName2, "claude.txt")
perplexity = connect(file_path3, myStates3, fileName3, "perplexity.txt")
chatgpt = connect(file_path4, myStates4, fileName4, "chatgpt.txt")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# loads markdown files, possibly want to create a pipeline for this.
def doc_loader(file_path):
  loader = UnstructuredMarkdownLoader(
      file_path,
      mode="elements",
      strategy="fast")
  documents = loader.lazy_load()
  docs = []
  for doc in documents:
    docs.append(doc)
  print(docs[0])
  print(docs[0].metadata)
  return docs

docs = doc_loader(file_path)
# user_docs = doc_loader(user_policy) # temp doc
# user_docs = doc_loader("deepseek.txt")
# user_docs = doc_loader("claude.txt")
user_docs = doc_loader("perplexity.txt")
# user_docs = doc_loader("chatgpt.txt")

page_content='---- Page 0 ---- NIST AI 100-1 Artificial Intelligence Risk Management Framework (AI RMF 1.0)' metadata={'source': 'drive/MyDrive/datasci266/finalProject/references/NIST.AI.100-1.md', 'languages': ['eng'], 'file_directory': 'drive/MyDrive/datasci266/finalProject/references', 'filename': 'NIST.AI.100-1.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-05T21:27:37', 'category': 'UncategorizedText', 'element_id': 'a2fb64997515c7b058cd428590961861'}
{'source': 'drive/MyDrive/datasci266/finalProject/references/NIST.AI.100-1.md', 'languages': ['eng'], 'file_directory': 'drive/MyDrive/datasci266/finalProject/references', 'filename': 'NIST.AI.100-1.md', 'filetype': 'text/markdown', 'last_modified': '2025-03-05T21:27:37', 'category': 'UncategorizedText', 'element_id': 'a2fb64997515c7b058cd428590961861'}
page_content='INTERNAL AI POLICY STATE OF HAWAII Effective Date: March 25, 2025 | Approved By: Hawaii Artificial Intelligence Advisory Council' metadata={'source': 'perpl

In [9]:
# split doc into chuncks
def make_retriever(docs, chunk_size=1000, chunk_overlap=100, col_name="default"):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  split_docs = text_splitter.split_documents(docs)

  filtered_docs = []
  for doc in split_docs:
    if isinstance(doc, Document) and hasattr(doc, 'metadata'):
      clean = {k: v for k, v in doc.metadata.items() if isinstance(v, (str, int, float, bool))}
      #filtered_docs.append(Document(page_content=doc.page_content, metadata=clean))
      filtered_docs.append(Document(page_content=doc.page_content))
# update embeddings param to the one we end up using.
  vectorStore = Chroma.from_documents(
      documents=filtered_docs,
      collection_name=col_name,
      embedding=GPT4AllEmbeddings(),
      persist_directory="./"+col_name+"db"
  )

  retriever = vectorStore.as_retriever()
  return retriever

retriever = make_retriever(docs, col_name="golden")
user_retriever = make_retriever(user_docs, col_name="user")

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 120MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 561MiB/s]


In [ ]:
# make document md converter

import os
import pymupdf4llm
import pathlib

dir_path = "drive/MyDrive/datasci266/finalProject/references/"
files = ["NIST.AI.100-1.pdf","UCB_Guidelines_for_the_Use_of_Generative_AI.md"]

def make_md(dir_path, files):
  md_text = pymupdf4llm.to_markdown(dir_path+files[0])
  fname = files[0].replace(".pdf",".md")
  with open(dir_path+fname) as f:
    f.write(md_text)


In [10]:
from google.colab import userdata
import os
from huggingface_hub import snapshot_download
token = userdata.get('HF_TOKEN')
#TOKEN = os.environ['HF_TOKEN']
# path = snapshot_download(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", cache_dir="Llama-2-8B", use_auth_token=token)


We'll also take advantage of a Hugging Face abstraction called a pipeline.  It is an easy way of experimenting with a model in inference mode.  We'll use that here to experiment with prompts (and possibly some hyperparameters) to imporve the quality of our results.

It takes a while to load this model -- on the order of ten minutes -- but once it is loaded you can keep reusing the loaded model and improve your prompt.



In [11]:
"""
Initialize the pipeline with bitsandbytes quantization
"""
# Configure bitsandbytes for 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Initialize pipeline
model_id = "meta-llama/Llama-3.1-8B-Instruct"
#model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

summarizer = pipeline("text-generation", model=model_id, device_map="auto", trust_remote_code=True, token=token, model_kwargs={"torch_dtype": torch.bfloat16, "quantization_config": quantization_config})



config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


As a reminder, here's the record we're dealing with.

Let's just generate one summary so we can see what it looks like

In [12]:
# just need to create the retreiver function
def get_docs(q, retriever):
  pg_cont = []
  docs = retriever.get_relevant_documents(q)
  for i in range(2):
    pg_cont.append(docs[i].page_content)
  #print(f"docs: \n{docs[0:2]}")
  print(f"pg content \n{pg_cont}")
  #return {'documents': docs[0:2], "question": q}
  return pg_cont


# relevence scoring function
def get_score(question, document):
  question = question
  document = document
  print(f"document: \n{document}")

  prompt = [
              {"role": "system", "content": """are a grader assessing the relevance of a retrieved document to a user's question.
              if the document contains keywords related to the user question, grade it as relevant. It does not need to be a stringent
              test. The goal is to filter out erroneous retrievals. Give a binary score 'yes' or 'no' score to indicate if the document
              is relevant or not. Provide the binary score as a JSON with a single key 'score' and no preamble or explaination."""},
              {"role": "user", "content": f"user question: {question}"},
              {"role": "user", "content": f"retrieved document: {document}"}
          ]



  outputs = summarizer(
    prompt,
    max_new_tokens=512,
    temperature = 0.3,
  )

  summary = outputs[0]["generated_text"][-1]
  print(f"\nscore: {summary['content']}")
  return summary

# answer the question using the relevent doc
# need a retreiver for the user policy and a retriever for the golden docs.
def answer_q(q, d):
  question = q
  document = d

  prompt = [
      {"role":"system", "content":"""you are an assistant helping answer user questions on a retrieved document.
      the goal is to generate an answer to their question using the retrieved document as a supporting source of information. """},
      {"role":"user", "content":f"user question: {question}"},
      {"role": "user", "content": f"retrieved document: {document}"}]

  outputs = summarizer(
    prompt,
    max_new_tokens=512,
    temperature = 0.3,
  )

  summary = outputs[0]["generated_text"][-1]
  print(f"\nanswer: {summary['content']}")
  return summary


Now we put the peices together, starting with a user question (u_question) to retrieve documents that are relevent to the question.

rel_score returns JSON: {'score':'yes or no'}

generates an answer is yes.

In [ ]:
u_question = "How to use AI in a ethical and responsible way?"

ret_docs = get_docs(u_question, retriever)

# should put in a loop for each doc passed back?
rel_score = get_score(u_question, ret_docs)

if 'yes' in rel_score['content']:
  answer_q(u_question, ret_docs)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


docs: 
[Document(metadata={'filetype': 'text/markdown', 'source': 'drive/MyDrive/datasci266/finalProject/references/NIST.AI.100-1.md', 'last_modified': '2025-03-05T21:27:37', 'category': 'NarrativeText', 'filename': 'NIST.AI.100-1.md', 'element_id': 'c7f40f53afec7541153355c0ba5e8068', 'file_directory': 'drive/MyDrive/datasci266/finalProject/references'}, page_content='social responsibility, and sustainability. AI risk management can drive responsible uses and practices by prompting organizations and their internal teams who design, develop, and deploy AI to think more critically about context and potential or unexpected negative and positive impacts. Understanding and managing the risks of AI systems will help to enhance trustworthiness, and in turn, cultivate public trust. Page 1'), Document(metadata={'filename': 'NIST.AI.100-1.md', 'file_directory': 'drive/MyDrive/datasci266/finalProject/references', 'element_id': 'c7f40f53afec7541153355c0ba5e8068', 'last_modified': '2025-03-05T21:27

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



score: {"score": "yes"}

answer: According to the retrieved document, to use AI in an ethical and responsible way, consider the following key components:

1. **AI risk management**: Understanding and managing the risks of AI systems will help to enhance trustworthiness and cultivate public trust. This involves thinking critically about context and potential or unexpected negative and positive impacts.
2. **Human centricity**: AI systems should be designed with human values and needs in mind, prioritizing human well-being and dignity.
3. **Social responsibility**: AI development and use should align with intended aims and values, emphasizing social responsibility and sustainability.
4. **Proper controls**: Implementing proper controls can help mitigate and manage inequitable outcomes, ensuring that AI systems do not amplify, perpetuate, or exacerbate undesirable outcomes for individuals and communities.
5. **Responsible AI practices**: Aligning decisions about AI system design, develop

**Handling the two vector dbs** <br><br>

when initializing the vector db's, we have to list the persistent dir, which i have just made the collection name + db. They are stored in 2 different objects so this should cause any issues down the line.  <br>

After we get the relevent sections of the docs, we will have to make a pandas dataframe so that we can store them in a CSV to transfer over to the BERT notebook. will need to come up with a better way to coordinate this hand off.

In [13]:
# the question prompts for each critical topic.

govern = [""" Ensure all AI tools used in decision-making undergo regular GRC evaluations. """, """ Make AI-driven decisions transparent and reviewable by designated oversight teams. """]

map = [""" Maintain an up-to-date inventory of all AI systems and their respective use cases. """, """ Document AI development processes and confirm alignment with applicable local laws. """]

Measure = [""" Conduct regular tests on AI systems for fairness, accuracy, bias, and security compliance. """, """ Perform impact assessments to evaluate explainability and internal or public effects of AI tools. """]

Manage = [""" Establish a clear response plan for AI system failures or cyber incidents involving AI. """, """ Monitor AI system behavior and track user interactions for anomalies and misuse. """]



**transfer to a pandas dataframe**

In [14]:
import pandas as pd

index = ['gov0', 'gov1', 'map0', 'map1', 'mea0', 'mea1', 'man0', 'man1']
cols = ['user', 'golden']

gov_user_txt, gov_golden_txt, map_user_txt, map_golden_txt, mea_user_txt, mea_golden_txt, man_user_txt, man_golden_txt = [], [], [], [], [], [], [], []

# df = pd.DataFrame(columns=cols, index=index)
# govern
gov_user_txt = ["".join(get_docs(govern[0], user_retriever))]
gov_golden_txt = ["".join(get_docs(govern[1], user_retriever))]
gov_user_txt.append("".join(get_docs(govern[0], retriever)))
gov_golden_txt.append("".join(get_docs(govern[1], retriever)))

# map
map_user_txt = ["".join(get_docs(map[0], user_retriever))]
map_golden_txt = ["".join(get_docs(map[1], user_retriever))]
map_user_txt.append("".join(get_docs(map[0], retriever)))
map_golden_txt.append("".join(get_docs(map[1], retriever)))

# measure
mea_user_txt = ["".join(get_docs(Measure[0], user_retriever))]
mea_golden_txt = ["".join(get_docs(Measure[1], user_retriever))]
mea_user_txt.append("".join(get_docs(Measure[0], retriever)))
mea_golden_txt.append("".join(get_docs(Measure[1], retriever)))

# manage
man_user_txt = ["".join(get_docs(Manage[0], user_retriever))]
man_golden_txt = ["".join(get_docs(Manage[1], user_retriever))]
man_user_txt.append("".join(get_docs(Manage[0], retriever)))
man_golden_txt.append("".join(get_docs(Manage[1], retriever)))

#print(f"user len: {len(user_txt)} \ngolden len {len(golden_txt)}")

df = pd.DataFrame([gov_user_txt, gov_golden_txt, map_user_txt, map_golden_txt, mea_user_txt, mea_golden_txt, man_user_txt, man_golden_txt,], columns=cols, index=index)

#### CHANGE THE FILE NAME!!!####
df.to_csv(f'perplexity_content.csv')
df.head()

<ipython-input-12-fc13280e61a2>:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(q)


pg content 
['We will conduct regular assessments to evaluate AI system performance and impact, using these evaluations to inform policy updates and improvements.', 'Regular reviews will be conducted to assess AI system performance and impact on constituents. These reviews will help identify areas for improvement and ensure that AI tools are meeting their intended objectives.']
pg content 
['Clear responsibilities will be designated within departments for AI-related decisions and actions. This will ensure that there are accountable individuals and teams for each AI-driven initiative.', 'Transparency will also involve regular reporting on the outcomes and impacts of AI-driven initiatives, ensuring that stakeholders are informed about how AI is influencing our operations and services.']
pg content 
['and improve performance can also increase levels of statistical uncertainty and cause issues with bias management, scientific validity, and reproducibility. • Higher degree of difficulty in 

,user,golden
gov0,We will conduct regular assessments to evaluat...,and improve performance can also increase leve...
gov1,Clear responsibilities will be designated with...,Issues that merit further consideration and re...
map0,Maintain a public AI inventory cataloging acti...,"such as predictions, recommenda- tions, or dec..."
map1,This policy is designed to be a living documen...,"(e.g., UX/UI design), governance experts, data..."
mea0,We will regularly audit AI systems for biases ...,the AI lifecycle can en- hance opportunities f...


In [15]:
df.tail()

,user,golden
map1,This policy is designed to be a living documen...,"(e.g., UX/UI design), governance experts, data..."
mea0,We will regularly audit AI systems for biases ...,the AI lifecycle can en- hance opportunities f...
mea1,Transparency: The practice of clearly communic...,"experience, perform human-centered evaluation ..."
man0,Publish annual transparency reports detailing ...,"such as predictions, recommenda- tions, or dec..."
man1,6.1 Auditing and Monitoring The AI Governance ...,"such as predictions, recommenda- tions, or dec..."


**Explainability Module** <br><br>

here, we import in the grades from the BERT semantic similarity analysis and have the meta llama model explain the results and offer feedback.

In [16]:
# protocol for ingesting AI document grade from other model
# gives a summary on the strengths and weaknesses based on the doc's score
# and the socring criteria, maybe pass the lowest categories?
import pandas as pd



def score_summary(scores, utxt):
   prompt = [
              {"role": "system", "content": """You are explaining the results of an evaluation done on a set of
              AI governance documents to a user. The goal is to summarize and explain the results, highlight
              the areas with the lowest scores and offer suggestions on improvments, make sure to say at least one good thing
              about the documet. Avoid using redunant language and try to be a concise as posible. The user will pass the
              the reults of the evaluation. There are four categories: govern, map, measure, and manage; each category has a
              percentage score with a plus or minus for neutral content. construct your feedback into points about the categories. """},
              {"role": "user", "content": f"user question: Summarize the evaluation results: {scores}"},
              {"role":"user", "content": f"user's policy that was graded: {utxt}"}]
   outputs = summarizer(
      prompt,
      max_new_tokens=512,
      temperature = 0.3
      )
   summary = outputs[0]["generated_text"][-1]
   print(f"\nanswer: {summary['content']}")
   return summary

# import the scores in from the BERT model.
temp_score_df = pd.read_csv('perplexity_scores.csv', index_col=None)
scoredf = temp_score_df.drop(columns=['Unnamed: 0'])
# scoredf.head()
ut = list(df['user'])
score_sum = score_summary(scoredf, ut)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



answer: **Summary of Evaluation Results:**

The AI governance policy received a mixed evaluation score. Here are the key findings:

**1. Governance (33.33% +/- 66.66%):**
The governance category scored low, indicating that the policy may not clearly define roles and responsibilities, accountability, and decision-making processes. Suggestions for improvement include:
- Clarifying the structure and responsibilities of the AI Governance Team.
- Establishing clear lines of authority and decision-making processes.
- Providing more detailed information on accountability and consequences for non-compliance.

**2. Map (33.33% +/- 33.33%):**
The mapping category scored low, indicating that the policy may not provide a comprehensive overview of AI systems, tools, and initiatives. Suggestions for improvement include:
- Developing a more detailed and up-to-date inventory of AI tools and systems.
- Establishing clear criteria for categorizing and labeling AI systems.
- Providing more information o

I am using this model to automate the labeling of this dataset for me.


In [ ]:


# answer the question using the relevent doc
def auto_labeler(text):

  prompt = [
      {"role":"system", "content":"""you are an assistant helping to label a dataset for a BERT model doing sentiment analysis.
      There are 3 categories to pick from, each one has an explaination and an example of what would fit in each given category.

      Theses are the categories and their explainations:
      Active: the tone of a given section is conveying the actions that will be taken, things like “Decisions impacted by AI are reviewed by a human in the loop and have a sign off process.” In this example, it talks about what is currently actively being done.
      Neutral: a section with neutral tone might be something like definitions or explanation of roles. More focused on giving context, background, and defining items for common understanding. ex. "Generative Artificial Intelligence: For the purposes of this policy, Artificial Intelligence (AI) encompasses both Traditional AI and Generative AI."
      Incomplete: a section categorized as in-complete points to a vagueness to statements, lack of conviction, lack of action, or lack of consideration for potential events. An example of this would be giving a specific statement about something like requirements but does not include the actual information needed to understand the requirements; or mentioning a process/task to be done but then doesn’t explain or give any additional relevant information. ex. "Disclose the use of AI when necessary to the proper audience."

      In your response, only return the "category name" and "1" indicating that this is the best fitting category for the current piece of text.
      Provide the category name as a JSON with a single key 'category' and no preamble or explaination.

         """},
      {"role":"user", "content":f"user question: {text}"}]

  outputs = summarizer(
    prompt,
    max_new_tokens=512,
    temperature = 0.3,
  )

  summary = outputs[0]["generated_text"][-1]
  print(f"\nanswer: {summary['content']}")
  return summary






In [ ]:

# import pandas as pd
# df = pd.read_csv('output.csv')
# #print(df.tail())
# responses = []

# for i in df['text']:
#   resp = auto_labeler(i)
#   print(resp)
#   responses.append(resp)


FileNotFoundError: [Errno 2] No such file or directory: 'output.csv'

In [ ]:

# df['response'] = responses

# df.to_csv('resp2.csv', mode='a', header=True, index=True)

